In [ ]:
import numpy as np
import importlib
from datetime import datetime

from helpers import pre_processing_wlan_utils as preprocess_utils
from helpers import classifier_wlan_spectral_utils as classifier_utils
from helpers import tr_models as tr_models

In [ ]:
seed = 42
task = "frames"
label = preprocess_utils.label_index[task]
num_classes = preprocess_utils.num_classes[task]
print("Label id: ", label)
print("Num classes in that label: ", num_classes)
max_iq_samples = [3000, 5000]
padding = 'post'
dataset_folder = '../../dataset/waveforms/'
load_model=False
load_dir='../../models/'
load_file = '22042020175144_saved_wavenet_classifier'
prefix ='cnn_wlan_classifier

In [ ]:
filters=32

In [ ]:
now = datetime.now()
datenow = now.strftime('%d%m%y%H%M%S')

In [ ]:
importlib.reload(tr_models)
importlib.reload(preprocess_utils)
importlib.reload(classifier_utils)

In [ ]:
dataset_filename = 'waveforms_SNR_16042020_2G_n_unknown_unknown_v2WLAN_CLASS_wlan-frame_balanced.mat'
Xraw, Yraw = classifier_utils.get_raw_xy_spectrum(dataset_folder,dataset_filename)

In [ ]:
acc_results=[]
for num_iq_seq in max_iq_samples:
    time.sleep(2)
    print("Padding/Truncating sequence with max lenght: ", str(num_iq_seq))
    X = classifier_utils.pad_or_trunc_x_and_scale(Xraw, num_iq_seq, padding, scale=True)
    time.sleep(2)
    
    print("Oneshot labels")
    Y = classifier_utils.get_one_hot_labels(Yraw, num_classes, label)
    
    print("Performing partitioning of data")
    X_train, X_val, X_test, Y_train, Y_val, Y_test = classifier_utils.get_xy_4_training(X,Y,seed)
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
    X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], X_val.shape[2], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
    print(X_train.shape,X_val.shape, X_test.shape)
    
    for t in [1,2]:
        print("Creating the model")
        cnn = tr_models.CNNClassifier((2,num_iq_seq), num_classes, filters = filters, prefix=prefix)
        print("Model created")

        print("Model start training")
        history = cnn.fit(X_train, Y_train, validation_data = (X_val, Y_val), batch_size = 16, save=True, epochs = 20)

        print("Training finished")
        test_results = cnn.evaluate(X_test, Y_test)
        acc_results.append([num_iq_seq, test_results])

        print(num_iq_seq, test_results)

        #Save conf matrix
        file_out_conf_matrix = str(datenow)+"_"+prefix+"_iq_samples_"+str(num_iq_seq)+"_iter_"+str(t)+"_frames_conf_matrix_cnn.pdf"
        Y_pred=np.argmax(cnn.predict(X_test),1)
        Y_true=np.argmax(Y_test,1)
        classifier_utils.plot_confusion_matrix_mc(Y_true, Y_pred, ["Mgmt","Ctrl","Data"], file_out_conf_matrix)

In [ ]:
from numba import cuda
cuda.select_device(0)
cuda.close()